In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger import *

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
from pathlib import Path

In [7]:
from typing import Dict

In [8]:
caseName : str = "top_down_stair_2d_produce_dataset"

In [9]:
math = cp
spatialStep : float = .003
length : float = 1
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
minimumPointCount : int = int(length / spatialStep)
print("Minimum Point Count: ", minimumPointCount)
pointCount : int = 400
assert(pointCount >= minimumPointCount)
potentialHeight : float = 50000
preRegionLength = .3
preRegionScalar = 2
defaultPacketStandardDeviation : float = preRegionLength / preRegionScalar
defaultPacketStartX : float = 1 - ((preRegionScalar / 2) * defaultPacketStandardDeviation)
print("Default Packet Start X: ", defaultPacketStartX)
packetStandardDeviation : float = preRegionLength / 5
packetStartX : float = .5
packetStartY : float = .65
packetWaveNumber : float = 150 * np.pi
barrierCount : int = 3
postRegionLength = .3
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print("Point Count: ", pointCount)
print("Post Region Length: ", postRegionLength)
print("Intra Region Length: ", intraRegionLength)
print("Pre Region Length: ", preRegionLength)
print("Barrier Length: ", barrierWidth)
print("Total Length: ", intraRegionLength + postRegionLength + preRegionLength)
print("Total Length Ratios: ", np.sum(constantPotentialLengthRatios))
barrierPixels : float = (barrierWidth * barrierCount * pointCount)
assert((barrierPixels - int(barrierPixels) < math.finfo(float).eps))
barrierPixels : int = int(barrierPixels)
print("Barrier Pixels: ", barrierPixels)
frameCount : int = 750
backend = "numpy"

Minimum Point Count:  333
Default Packet Start X:  0.85
Point Count:  400
Post Region Length:  0.3
Intra Region Length:  0.4
Pre Region Length:  0.3
Barrier Length:  0.13333333333333333
Total Length:  1.0
Total Length Ratios:  1.0
Barrier Pixels:  160


In [10]:
wavePacketFunction = lambda position : makeWavePacket(
            position, 
            packetStartX * length, 
            packetStartY * length, 
            spatialStep, 
            packetStandardDeviation, 
            packetWaveNumber, 
            DimensionIndex.Y, 
            math = math
        )

In [11]:
potentials : List[List[float]] = [
        [0, 0, 0, 0, 0], 
        [0, 0, 0, 0, 0], 
        [0, 1, 2 / 3, 1 / 3, 0], 
        [0, 1, 1, 1, 0], 
        #[0, 1 / 3, 2 / 3, 1, 0], 
        #[0, 2 / 3, 1 / 3, 1, 0], 
        #[0, 1, 1 / 3, 2 / 3, 0], 
        #[0, 0, 0, 0, 0]
    ]

In [12]:
boxHeight : float = .2
boxYPosition : float = 0.0
totalLength : float = 0.0
constantBoxes : List[Rectangle2D] = []
for boxLength in constantPotentialLengthRatios: 
    constantBoxes.append(Rectangle2D(totalLength, boxYPosition, boxLength, boxHeight))
    totalLength += boxLength

In [13]:
constantBoxes 

[Rectangle2D(x=0.0, y=0.0, width=0.3, height=0.2),
 Rectangle2D(x=0.3, y=0.0, width=0.13333333333333333, height=0.2),
 Rectangle2D(x=0.43333333333333335, y=0.0, width=0.13333333333333333, height=0.2),
 Rectangle2D(x=0.5666666666666667, y=0.0, width=0.13333333333333333, height=0.2),
 Rectangle2D(x=0.7, y=0.0, width=0.3, height=0.2)]

In [14]:
print(potentials[0])
potentials[0].reverse()
print(potentials[0])

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]


In [15]:
profiles : List[SimulationProfile] = constantSimulationProfiles(
        wavePacketFunction, 
        spatialStep, 
        temporalStep, 
        length, 
        constantBoxes, 
        potentials, 
        potentialHeight, 
        pointCount, 
        simulateControl = False, 
        gpuAccelerated = True, 
        edgeBound = True, 
        constantPotentialFunction = axisAlignedBlocks, 
        defaultMatrixSolveMethod = lambda profile, operator, independantTerms : solveMatrixApproximate(profile, operator, independantTerms, 1e-4)
    )

In [16]:
allData, simulations, _ = recordConstantRegionSimulations(
        profiles, 
        frameCount, 
        caseName, 
        [
            constantPotentialLengthRatios[0], 
            sum(constantPotentialLengthRatios[1:4]), 
            constantPotentialLengthRatios[4]
        ], #constantPotentialLengthRatios, 
        False, 
        constantRegionLabels = ["Post", "Intra", "Pre"], #["Post", "3rdStair", "2ndStair", "1stStair", "Pre"], 
        showBar = True, 
        showFPS = True, 
        showTotalTime = True, 
        recordAllRegionVideos = False, 
        discardSimulations = False
    )

[====================================================================================================]
[----------------------------------------------------------------------------------------------------]
Total Time:  34.07633248000002
Frames Per Second:  23.027578728143247
Saving Video of Post
Saving Video of Intra
Saving Video of Pre
[====================================================================================================]
[---------------------------------------------------------------------------------

OutOfMemoryError: Out of memory allocating 9,591,808 bytes (allocated so far: 5,282,652,160 bytes).

In [ ]:
for value in allData.keys(): 
    #print(type(allData[value]))
    if type(allData[value]) is cp.ndarray: 
        allData[value] = allData[value].get()

In [ ]:
pd.DataFrame(allData).to_csv(str(Path.cwd() / caseName / (caseName + ".csv")))

In [ ]:
import hf5py as h5

In [ ]:
simulations

In [ ]:
simulations[0]

In [ ]:
#for simulation in simulations: 
    #for probability in probabilities: 
        